In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import imodelsx.metrics
import numpy as np
sys.path.append('../experiments/')
import os
import pandas as pd
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from typing import Dict, List
from dict_hash import sha256
import numpy as np

import imodelsx.treeprompt.stump

from tprompt.compiler.evaluator import PromptHooker, modify_activations
from tprompt.compiler import compiling

# Load prompt vector

In [2]:
# hyperparams
checkpoint = 'gpt2'
prompts = [
    " This review of a movie is",
    " Positive or Negative? The movie was",
    " The sentiment of the movie was",
    # " The plot of the movie was really",
    " The acting in the movie was",

]
prompt_at_start_or_end = 'start'
# note: also requires specifying the layer for the hook below

In [3]:
avg_soft_prompt = compiling.get_avg_soft_prompt(checkpoint, prompts)

# Evaluate accuracies

In [4]:
dset_train = datasets.load_dataset('rotten_tomatoes')['train']
dset_train = dset_train.select(np.random.choice(
    len(dset_train), size=100, replace=False))
# dset_val = datasets.load_dataset('rotten_tomatoes')['validation']
# dset_val = dset_val.select(np.random.choice(
#     len(dset_val), size=100, replace=False))

  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
verbalizer = {0: " Negative.", 1: " Positive."}

m = PromptHooker(
    checkpoint=checkpoint,
    # this should probably be the prompt with the max num tokens?
    prompts=[prompts[0]],
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    hook_weights=avg_soft_prompt,
    prompt_at_start_or_end='end',
    prompt_template="{example}{prompt}",
)
m.fit(dset_train["text"], dset_train["label"])
acc_avg = m.prompt_accs_[0]

print('------------------------')
m = PromptHooker(
    checkpoint=checkpoint,
    prompts=prompts,
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    hook_weights=None,
    prompt_at_start_or_end='end',
)
m.fit(dset_train["text"], dset_train["label"])
accs0 = deepcopy(m.prompt_accs_)
print('**Averaged** ->', acc_avg)
for i, prompt in enumerate(prompts):
    print(i, prompt, '->', accs0[i])

Prompt 0:  This review of a movie is


------------------------
Prompt 0:  This review of a movie is


Prompt 1:  Positive or Negative? The movie was


Prompt 2:  The sentiment of the movie was


Prompt 3:  The acting in the movie was


**Averaged** -> 0.56
0  This review of a movie is -> 0.49
1  Positive or Negative? The movie was -> 0.49
2  The sentiment of the movie was -> 0.52
3  The acting in the movie was -> 0.5


# Unit test

In [13]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained(checkpoint)
tok_lens = [len(tok.encode(prompt)) for prompt in prompts]

In [14]:
tok_lens

[6, 7, 6, 6]

In [15]:
m = PromptHooker(
    checkpoint=checkpoint,
    # 3 different prompts with same len
    prompts=[prompts[0], prompts[2], prompts[3]],
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    hook_weights=avg_soft_prompt,
    prompt_at_start_or_end='end',
    prompt_template="{example}{prompt}",
)
m.fit(dset_train["text"], dset_train["label"])

Prompt 0:  This review of a movie is


Prompt 1:  The sentiment of the movie was


Prompt 2:  The acting in the movie was


PromptHooker(cache_prompt_features_dir=None, checkpoint='gpt2',
             hook_weights=tensor([[[ 0.0242, -0.1042,  0.0437,  ..., -0.0471,  0.0455,  0.0430],
         [-0.0821, -0.1882,  0.0493,  ..., -0.0688,  0.0849,  0.1223],
         [ 0.0759, -0.1216,  0.0096,  ..., -0.0654,  0.0015,  0.0660],
         ...,
         [-0.0548, -0.0564,  0.1641,  ..., -0.0132,  0.0767, -0.0293],
         [ 0.0336, -0.0667,  0.3518,  ..., -0.0217,  0.0272,  0.1439],
         [-0.0497, -0.0910,  0.2781,  ...,  0.1334, -0.0300, -0.0691]]],
       device='cuda:0'),
             prompts=[' This review of a movie is',
                      ' The sentiment of the movie was',
                      ' The acting in the movie was'])

In [16]:
m.prompt_accs_

[0.56, 0.56, 0.56]